In [ ]:
%load_ext autoreload
%autoreload 2

In [2]:
import pyspark.sql.functions as F
import pyspark.sql.types as T
from pyspark.sql import SparkSession

In [3]:
from shared.schema import DatasetSchema

DATASET = DatasetSchema.load_schema('email-Eu-core')
DATASET.save_schema()

In [4]:
spark = (SparkSession.builder
         .appName(f'{DATASET}')
         .config('spark.sql.legacy.timeParserPolicy', 'LEGACY')
         .config("spark.executor.memory", "8g")
         .config("spark.driver.memory", "8g")
         .config("spark.memory.offHeap.enabled", True)
         .config("spark.memory.offHeap.size", "16g")
         .getOrCreate())

22/01/22 22:11:02 WARN Utils: Your hostname, megatron resolves to a loopback address: 127.0.1.1; using 192.168.1.89 instead (on interface enp7s0)
22/01/22 22:11:02 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/01/22 22:11:02 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/01/22 22:11:03 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/01/22 22:11:03 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
22/01/22 22:11:03 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.


In [5]:
schema = T.StructType([
    T.StructField('src', T.IntegerType(), False),
    T.StructField('dst', T.IntegerType(), False),
])

df = (
    spark.read.csv(DATASET.raw_str('email-Eu-core.txt'), sep=' ', header=False, schema=schema, mode='DROPMALFORMED')
)
print(df.head(5))
print(df.count())

[Row(src=0, dst=1), Row(src=2, dst=3), Row(src=2, dst=4), Row(src=5, dst=6), Row(src=5, dst=7)]
25571


In [6]:
df_nodes = (
    df.select(F.col('src').alias('id'))
        .union(df.select(F.col('dst').alias('id')))
        .distinct()
)
df_nodes.head(5)

[Row(id=148), Row(id=471), Row(id=496), Row(id=463), Row(id=833)]

In [7]:
df_edges = (
    df.select(F.col('src'), F.col('dst'))
        .distinct()
)
df_edges.head(5)

[Row(src=108, dst=112),
 Row(src=280, dst=129),
 Row(src=268, dst=106),
 Row(src=236, dst=238),
 Row(src=147, dst=105)]

In [8]:
df_nodes.write.parquet(DATASET.processed_str('nodes_User'), mode='overwrite')
df_edges.write.parquet(DATASET.processed_str('edges_COMMUNITCATES'), mode='overwrite')

In [9]:
from shared.schema.graph import GraphSchema, NodeSchema, EdgeSchema

(
    GraphSchema()
        .add_node_schema('User', NodeSchema.from_spark(df_nodes.schema, label='id'))
        .add_edge_schema('COMMUNITCATES', EdgeSchema.from_spark(df_edges.schema, source_type='User', target_type='User', directed=False))
        .save_schema(DATASET.processed())
)

GraphSchema(_path=PosixPath('/dd_volume/Development/Python/Thesis/code/datasets/data/processed/email-Eu-core'), nodes={'User': NodeSchema(_type='User', _schema=..., label='id', properties={'id': GraphProperty(_name='id', dtype=DType(atomic=<DTypeAtomic.INT: 'int'>, array=False))}, dynamic=None)}, edges={'COMMUNITCATES': EdgeSchema(_type='COMMUNITCATES', _schema=..., label=None, properties={'src': GraphProperty(_name='src', dtype=DType(atomic=<DTypeAtomic.INT: 'int'>, array=False)), 'dst': GraphProperty(_name='dst', dtype=DType(atomic=<DTypeAtomic.INT: 'int'>, array=False))}, dynamic=None, source_type='User', target_type='User', directed=False)})

In [11]:

from shared.graph import read_comlist, write_comlist

comlist = read_comlist(DATASET.raw_str('email-Eu-core-department-labels.txt'), delimiter=' ')
write_comlist(comlist, DATASET.processed_str('ground_truth.ncomlist'))